In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import os
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler


from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller


import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import datetime
from datetime import date, timedelta
e_date = datetime.datetime.strptime('2020-06-20', '%Y-%m-%d')
i_date = datetime.datetime.strptime('2020-04-01', '%Y-%m-%d')

data_path = os.path.join('..','OD')

In [2]:
delta = e_date - i_date

target_days = []
for i in range(delta.days + 1):
    day = i_date + timedelta(days=i)
    target_days.append(day)

In [3]:
def read_OD_fn(source_type):
    ods = {}
    for date in tqdm_notebook(target_days):
        od_date= pd.read_csv(os.path.join(data_path, 'date_{}_OD_{}.csv'.format(source_type,date.strftime('%Y-%m-%d'))), 
                            index_col=0)
        ods[date] = od_date
    return ods

In [4]:
ine_ods = read_OD_fn('INE')

  0%|          | 0/81 [00:00<?, ?it/s]

In [5]:
twt_ods = read_OD_fn('TWT')

  0%|          | 0/81 [00:00<?, ?it/s]

# Serializar los datos

In [6]:
lista_twt1 = []
lista_twt2 = []
lista_twt3 = []
lista_twt4 = []
lista_twt5 = []

for date in tqdm_notebook(target_days):
    lista_twt1.append(twt_ods[date].loc['58MA','total'])
    lista_twt2.append(twt_ods[date].loc['156B','total'])
    lista_twt3.append(twt_ods[date].loc['34CZ','total'])
    lista_twt4.append(twt_ods[date].loc['165V','total'])
    lista_twt5.append(twt_ods[date].loc['13VI','total'])
    
    
lista_ine1 = []
lista_ine2 = []
lista_ine3 = []
lista_ine4 = []
lista_ine5 = []

for date in tqdm_notebook(target_days):
    lista_ine1.append(ine_ods[date].loc['58MA','total'])
    lista_ine2.append(ine_ods[date].loc['156B','total'])
    lista_ine3.append(ine_ods[date].loc['34CZ','total'])
    lista_ine4.append(ine_ods[date].loc['165V','total'])
    lista_ine5.append(ine_ods[date].loc['13VI','total'])


  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

In [7]:
df_twt1 = pd.DataFrame (lista_twt1, columns = ['Viajes'], index= target_days)
df_ine1 = pd.DataFrame (lista_ine1, columns = ['Viajes'], index= target_days)

df_twt2 = pd.DataFrame (lista_twt2, columns = ['Viajes'], index= target_days)
df_ine2 = pd.DataFrame (lista_ine2, columns = ['Viajes'], index= target_days)

df_twt3 = pd.DataFrame (lista_twt3, columns = ['Viajes'], index= target_days)
df_ine3 = pd.DataFrame (lista_ine3, columns = ['Viajes'], index= target_days)

df_twt4 = pd.DataFrame (lista_twt4, columns = ['Viajes'], index= target_days)
df_ine4 = pd.DataFrame (lista_ine4, columns = ['Viajes'], index= target_days)

df_twt5 = pd.DataFrame (lista_twt5, columns = ['Viajes'], index= target_days)
df_ine5 = pd.DataFrame (lista_ine5, columns = ['Viajes'], index= target_days)

In [8]:
lista_twt = []
lista_ine = []

df_twt = df_twt1 + df_twt2 + df_twt3 + df_twt4 + df_twt5
df_ine = df_ine1 + df_ine2 + df_ine3 + df_ine4 + df_ine5

In [9]:
df_twt.head()

,Viajes
2020-04-01,11
2020-04-02,3
2020-04-03,14
2020-04-04,5
2020-04-05,13


In [10]:
df_ine.head()

,Viajes
2020-04-01,16278
2020-04-02,16895
2020-04-03,18153
2020-04-04,13180
2020-04-05,11805


In [11]:
del twt_ods
del ine_ods

# Visualización de datos

In [12]:
print(df_twt.index.min())
print(df_ine.index.max())

2020-04-01 00:00:00
2020-06-20 00:00:00


In [13]:
print(len(df_twt['2020']))

81


In [14]:
print(len(df_ine['2020']))

81


In [15]:
df_twt.describe()

,Viajes
count,81.000000
mean,17.518519
std,8.148790
min,0.000000
25%,11.000000
50%,16.000000
75%,24.000000
max,34.000000


In [16]:
df_ine.describe()

,Viajes
count,81.000000
mean,30777.061728
std,13658.229593
min,10416.000000
25%,18843.000000
50%,30457.000000
75%,42480.000000
max,55828.000000


In [17]:
### Cointegration test

trips_df = df_twt.copy()
trips_df['ine_trips']= df_ine['Viajes']
trips_df.head()

,Viajes,ine_trips
2020-04-01,11,16278
2020-04-02,3,16895
2020-04-03,14,18153
2020-04-04,5,13180
2020-04-05,13,11805


In [18]:
def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    
    def adjust(val, length= 6): 
        return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)


cointegration_test(trips_df)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Viajes ::  12.92     > 12.3212   =>   True
ine_trips ::  3.72      > 4.1296    =>   False


### Granger test

In [19]:
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(trips_df, variables = trips_df.columns)

,Viajes_x,ine_trips_x
Viajes_y,1.0000,0.0
ine_trips_y,0.0014,1.0


### Check stationarity

In [20]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

adfuller_test(trips_df['Viajes'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.4583
 No. Lags Chosen       = 6
 Critical value 1%     = -3.522
 Critical value 5%     = -2.901
 Critical value 10%    = -2.588
 => P-Value = 0.554. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


In [21]:
adfuller_test(trips_df['ine_trips'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 0.9935
 No. Lags Chosen       = 6
 Critical value 1%     = -3.522
 Critical value 5%     = -2.901
 Critical value 10%    = -2.588
 => P-Value = 0.9942. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


In [22]:
df_train, df_test = trips_df[0:-7], trips_df[-7:]

In [23]:
# Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape) 

(74, 2)
(7, 2)


In [24]:
trips_df_differenced = df_train.diff().dropna()

adfuller_test(trips_df_differenced['Viajes'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -6.9218
 No. Lags Chosen       = 5
 Critical value 1%     = -3.532
 Critical value 5%     = -2.906
 Critical value 10%    = -2.59
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [25]:
adfuller_test(trips_df_differenced['ine_trips'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -13.7441
 No. Lags Chosen       = 5
 Critical value 1%     = -3.532
 Critical value 5%     = -2.906
 Critical value 10%    = -2.59
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


### Select order for VAR Model

In [26]:
trips_df_differenced.columns

Index(['Viajes', 'ine_trips'], dtype='object')

In [27]:
model = VAR(trips_df_differenced)

In [28]:
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,22.21,22.28,4.426e+09,22.24
1,22.09,22.30,3.922e+09,22.17
2,22.05,22.40,3.782e+09,22.19
3,21.96,22.44,3.443e+09,22.15
4,21.98,22.60,3.514e+09,22.22
5,21.45,22.21,2.089e+09,21.75
6,20.74*,21.64*,1.032e+09*,21.09*
7,20.78,21.82,1.080e+09,21.19
8,20.84,22.02,1.158e+09,21.30
9,20.92,22.23,1.265e+09,21.43


Best lag is set to 7 days

In [29]:
model_fitted = model.fit(7)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 21, Sep, 2021
Time:                     11:40:40
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    21.6643
Nobs:                     66.0000    HQIC:                   21.0623
Log likelihood:          -839.376    FPE:                9.62586e+08
AIC:                      20.6690    Det(Omega_mle):     6.39083e+08
--------------------------------------------------------------------
Results for equation Viajes
                  coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------
const                0.024427         1.012540            0.024           0.981
L1.Viajes           -0.715878         0.133773           -5.351           0.000
L1.ine_trips         0.000262         0.000232            1.129

### Forecast test

In [30]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100


def compute_metrics_fn(y_valid, y_hat):
    mae_ = mean_absolute_error(y_valid, y_hat)
    mse_ = mean_squared_error(y_valid, y_hat)
    rmse_ = mean_squared_error(y_valid, y_hat, squared = False)
    cvrmse_ = rmse_/np.mean(y_valid)*100 # it is a percentage
    mape_ = mape(y_valid, y_hat)
    
    return mae_, mse_, rmse_, cvrmse_, mape_

trips_df.shape

(81, 2)

In [31]:
ine_true = []
ine_hat = []
for i in range(60, 81-7):
    X= trips_df.iloc[0:i]
    X_differenced = X.diff().dropna()
    
    y =trips_df.iloc[i:i+7]
    y_true_ine = [v[1] for v in y.values]
    
    model = VAR(X_differenced.iloc[:-7])
    model_fitted = model.fit(7)
    #print(len(X_differenced.iloc[-7:]))
    y_hat = model_fitted.forecast(y=X_differenced.values[-7:], steps=7)
    #print(y_hat)
    
    df_forecast = pd.DataFrame(y_hat, index=trips_df.iloc[i:i+7].index, columns=trips_df.columns + '_1d')

    columns = trips_df.columns
    for col in columns:  
        df_forecast[str(col)+'_forecast'] = X[col].iloc[-1] + df_forecast[str(col)+'_1d'].cumsum()

    #print(X.iloc[-1],df_forecast, trips_df.iloc[i:i+7])
    y_hat_ine = list(df_forecast['ine_trips_forecast'].values)
    
    ine_true = ine_true + y_true_ine
    ine_hat = ine_hat + y_hat_ine
    #print(y_hat, y, y_true_ine, y_hat_ine)

testScore_MAE, testScore_MSE, testScore_RMSE, testScore_CVRMSE, testScore_MAPE  =  compute_metrics_fn(ine_true, ine_hat)

print('Resultado del test : %.2f MAE' % ( testScore_MAE))
print('Resultado del test: %.2f MSE' % ( testScore_MSE))
print('Resultado del test: %.2f RMSE' %  (testScore_RMSE))
print('Resultado del test: %.2f CVRMSE' % ( testScore_CVRMSE))
print('Resultado del test: %.2f MAPE \n' % (testScore_MAPE))

Resultado del test : 3367.37 MAE
Resultado del test: 17924872.92 MSE
Resultado del test: 4233.78 RMSE
Resultado del test: 9.20 CVRMSE
Resultado del test: 7.87 MAPE 



In [32]:
for i in range(0,7):
    y_true = ine_true[i::7]
    y_hat = ine_hat[i::7]
    testScore_MAE, testScore_MSE, testScore_RMSE, testScore_CVRMSE, testScore_MAPE  =  compute_metrics_fn(y_true, y_hat)

    print('Resultado del test dia %d: %.2f MAE' % (i, testScore_MAE))
    print('Resultado del test dia %d: %.2f MSE' % (i, testScore_MSE))
    print('Resultado del test dia %d: %.2f RMSE' % (i, testScore_RMSE))
    print('Resultado del test dia %d: %.2f CVRMSE' % (i, testScore_CVRMSE))
    print('Resultado del test dia %d: %.2f MAPE \n' % (i, testScore_MAPE))

Resultado del test dia 0: 2839.99 MAE
Resultado del test dia 0: 15881667.32 MSE
Resultado del test dia 0: 3985.18 RMSE
Resultado del test dia 0: 8.93 CVRMSE
Resultado del test dia 0: 7.31 MAPE 

Resultado del test dia 1: 2460.34 MAE
Resultado del test dia 1: 9451472.53 MSE
Resultado del test dia 1: 3074.32 RMSE
Resultado del test dia 1: 6.82 CVRMSE
Resultado del test dia 1: 5.85 MAPE 

Resultado del test dia 2: 3526.69 MAE
Resultado del test dia 2: 18537711.31 MSE
Resultado del test dia 2: 4305.54 RMSE
Resultado del test dia 2: 9.43 CVRMSE
Resultado del test dia 2: 7.99 MAPE 

Resultado del test dia 3: 3770.25 MAE
Resultado del test dia 3: 21734152.74 MSE
Resultado del test dia 3: 4661.99 RMSE
Resultado del test dia 3: 10.12 CVRMSE
Resultado del test dia 3: 8.81 MAPE 

Resultado del test dia 4: 3791.13 MAE
Resultado del test dia 4: 20381412.14 MSE
Resultado del test dia 4: 4514.58 RMSE
Resultado del test dia 4: 9.71 CVRMSE
Resultado del test dia 4: 8.71 MAPE 

Resultado del test dia 5: